In [110]:
# https://preppindata.blogspot.com/2021/06/2021-week-26-rolling-weekly-revenue.html

import pandas as pd
import numpy as np
from datetime import date, timedelta, datetime 

### Input data

In [111]:
df = pd.read_csv(r'data\PD 2021 Wk 26 Input.csv')
df.head(10)

,Destination,Date,Revenue
0,London,01/01/2021,232572
1,London,02/01/2021,105610
2,London,03/01/2021,149849
3,London,04/01/2021,164463
4,London,05/01/2021,129130
5,London,06/01/2021,232296
6,London,07/01/2021,101116
7,London,08/01/2021,161349
8,London,09/01/2021,174858
9,London,10/01/2021,199301


### Create a data set that gives 7 rows per date (unless those dates aren't included in the data set). 
- ie 1st Jan only has 4 rows of data (1st, 2nd, 3rd & 4th)

In [112]:
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
ls_target_date = list(set(df['Date']))

In [113]:

df_all_rolling_days = pd.DataFrame()

for i in range(len(ls_target_date)):
    # create the 7 days range for each day
    start_date = ls_target_date[i] - timedelta(days=3)
    end_date = ls_target_date[i] + timedelta(days=3)
    rolling_days = pd.date_range(start_date,end_date,freq='d')

    # create a df holding the 7 days and target day
    df_rolling_days = pd.DataFrame()
    df_rolling_days['rolling days'] = rolling_days
    df_rolling_days['Target Date'] = ls_target_date[i]

    df_all_rolling_days = pd.concat([df_all_rolling_days, df_rolling_days])

df_all_rolling_days

,rolling days,Target Date
0,2021-01-18,2021-01-21
1,2021-01-19,2021-01-21
2,2021-01-20,2021-01-21
3,2021-01-21,2021-01-21
4,2021-01-22,2021-01-21
...,...,...
2,2021-01-16,2021-01-17
3,2021-01-17,2021-01-17
4,2021-01-18,2021-01-17
5,2021-01-19,2021-01-17


### Create the Rolling Week Total and Rolling Week Average per destination & Create the Rolling Week Total and Rolling Week Average for the whole data set
- Records that have less than 7 days data should remain included in the output

In [114]:
df_main = pd.merge(df_all_rolling_days,df, left_on='rolling days',right_on='Date',how='inner')

In [115]:
df_main_per_des=df_main.copy()
df_main_all=df_main.copy()
df_main_all['Destination'] = 'All'

#### Pull the data together for the previous two requirements
df_output = pd.concat([df_main_per_des,df_main_all])
df_output = df_output.groupby(['Destination','Target Date'])['Revenue'].agg(['sum']).reset_index()
df_output.rename(columns={'Target Date': 'Date', 'sum':'Rolling week Total'},inplace=True)
df_output['Rolling week Avg'] = df_output['Rolling week Total']/7
df_output

,Destination,Date,Rolling week Total,Rolling week Avg
0,All,2021-01-01,1945312,277901.714286
1,All,2021-01-02,2434642,347806.000000
2,All,2021-01-03,2876241,410891.571429
3,All,2021-01-04,3240012,462858.857143
4,All,2021-01-05,3114034,444862.000000
...,...,...,...,...
355,Perth,2021-03-27,703895,100556.428571
356,Perth,2021-03-28,719855,102836.428571
357,Perth,2021-03-29,574933,82133.285714
358,Perth,2021-03-30,494421,70631.571429


### Output the data

In [116]:
df_output.to_csv(r'output/2021-week26-output.csv')